In [ ]:
# projecteuler.net

# Problem 1
'''If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23.
Find the sum of all the multiples of 3 or 5 below 1000.'''

# Answer 1
sum([x for x in range(1000) if x%3==0 or x%5==0])

# Problem 2
'''
Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be:
1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ...
By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.'''

#Answer 2
l = [1, 2]
while l[-1] < 4000000:
    l.append(l[-1] + l[-2])
sum([i for i in l if i%2==0])

# Problem 3
'''The prime factors of 13195 are 5, 7, 13 and 29. What is the largest prime factor of the number 600851475143 ?'''

# Answer 3
def isPrime(n):
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            return False
    return True

def factors(n):
    l = []
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            l.append(x)
    return l

max([i for i in factors(600851475143) if isPrime(i)])

# Problem 4
'''A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is 9009 = 91 × 99.
Find the largest palindrome made from the product of two 3-digit numbers.'''

# Answer 4
def isPalendrome(n):
    str_n = str(n)
    for i in range(len(str_n)):
        if str_n[i] != str_n[len(str_n) - 1 - i]:
            return False
    return True

max([j for j in [i*j for i in range(1000) for j in range(1000)] if isPalendrome(j)])

# Problem 5
'''2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder.
What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?'''

# Answer 5
n = 1
while True:
    n += 1
    c = 20
    while c > 10:
        if n % c != 0:
            break
        c -= 1
    if c == 10:
        print(n)
        break


# Problem 6
'''The sum of the squares of the first ten natural numbers is, 12 + 22 + ... + 102 = 385
The square of the sum of the first ten natural numbers is, (1 + 2 + ... + 10)2 = 552 = 3025
Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is 3025 − 385 = 2640.
Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.'''

# Answer 6
f = lambda n: sum([x for x in range(n + 1)])**2 - sum([x**2 for x in range(n + 1)])
f(100)

# Problem 7
'''By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13. What is the 10 001st prime number?'''

# Answer 7
c, n = 0, 2
while True:
    if isPrime(n):
        c += 1
        if c == 10001:
            print(n)
            break
    n += 1
    
# Problem 8
'''
The four adjacent digits in the 1000-digit number that have the greatest product are 9 × 9 × 8 × 9 = 5832.

73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450

Find the thirteen adjacent digits in the 1000-digit number that have the greatest product. What is the value of this product?
'''

# Answer 8
l = [
'73167176531330624919225119674426574742355349194934',
'96983520312774506326239578318016984801869478851843',
'85861560789112949495459501737958331952853208805511',
'12540698747158523863050715693290963295227443043557',
'66896648950445244523161731856403098711121722383113',
'62229893423380308135336276614282806444486645238749',
'30358907296290491560440772390713810515859307960866',
'70172427121883998797908792274921901699720888093776',
'65727333001053367881220235421809751254540594752243',
'52584907711670556013604839586446706324415722155397',
'53697817977846174064955149290862569321978468622482',
'83972241375657056057490261407972968652414535100474',
'82166370484403199890008895243450658541227588666881',
'16427171479924442928230863465674813919123162824586',
'17866458359124566529476545682848912883142607690042',
'24219022671055626321111109370544217506941658960408',
'07198403850962455444362981230987879927244284909188',
'84580156166097919133875499200524063689912560717606',
'05886116467109405077541002256983155200055935729725',
'71636269561882670428252483600823257530420752963450'
]
def get_mult(l):
    m = 1
    for j in l:
        m *= int(j)
    return m
number = ''.join([i for i in l])
start, end = 0, 13
while end < len(number) + 1:
    new_number = number[start:end]
    out.append([new_number, get_mult(new_number)])
    start += 1
    end += 1
sorted(out, key=lambda x: x[-1], reverse=True)[0]

# Problem 9
'''A Pythagorean triplet is a set of three natural numbers, a < b < c, for which, a2 + b2 = c2
For example, 32 + 42 = 9 + 16 = 25 = 52.
There exists exactly one Pythagorean triplet for which a + b + c = 1000. Find the product abc.'''
 
# Answer 9
l = []
for comb in itertools.combinations(np.arange(1, 1000, 1), 3):
    if sum(comb) == 1000:
        l.append(comb)
        
a = [i for i in l if i[0]**2 + i[1]**2 == i[2]**2][0]
a[0] * a[1] * a[2]

# Problem 10
'''The sum of the primes below 10 is 2 + 3 + 5 + 7 = 17. Find the sum of all the primes below two million.'''

# Answer 10
l = []
for i in range(1, 2000000):
    if isPrime(i):
        l.append(i)
sum(l)

# Problem 11
'''
In the 20×20 grid below, four numbers along a diagonal line have been marked in red.

08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48

The product of these numbers is 26 × 63 × 78 × 14 = 1788696.

What is the greatest product of four adjacent numbers in the same direction (up, down, left, right, or diagonally) in the 20×20 grid?
'''

# Answer 11
l = [
[8,2,22,97,38,15,0,40,0,75,4,5,7,78,52,12,50,77,91,8],
[49,49,99,40,17,81,18,57,60,87,17,40,98,43,69,48,4,56,62,0],
[81,49,31,73,55,79,14,29,93,71,40,67,53,88,30,3,49,13,36,65],
[52,70,95,23,4,60,11,42,69,24,68,56,1,32,56,71,37,2,36,91],
[22,31,16,71,51,67,63,89,41,92,36,54,22,40,40,28,66,33,13,80],
[24,47,32,60,99,3,45,2,44,75,33,53,78,36,84,20,35,17,12,50],
[32,98,81,28,64,23,67,10,26,38,40,67,59,54,70,66,18,38,64,70],
[67,26,20,68,2,62,12,20,95,63,94,39,63,8,40,91,66,49,94,21],
[24,55,58,5,66,73,99,26,97,17,78,78,96,83,14,88,34,89,63,72],
[21,36,23,9,75,0,76,44,20,45,35,14,0,61,33,97,34,31,33,95],
[78,17,53,28,22,75,31,67,15,94,3,80,4,62,16,14,9,53,56,92],
[16,39,5,42,96,35,31,47,55,58,88,24,0,17,54,24,36,29,85,57],
[86,56,0,48,35,71,89,7,5,44,44,37,44,60,21,58,51,54,17,58],
[19,80,81,68,5,94,47,69,28,73,92,13,86,52,17,77,4,89,55,40],
[4,52,8,83,97,35,99,16,7,97,57,32,16,26,26,79,33,27,98,66],
[88,36,68,87,57,62,20,72,3,46,33,67,46,55,12,32,63,93,53,69],
[4,42,16,73,38,25,39,11,24,94,72,18,8,46,29,32,40,62,76,36],
[20,69,36,41,72,30,23,88,34,62,99,69,82,67,59,85,74,4,36,16],
[20,73,35,29,78,31,90,1,74,31,49,71,48,86,81,16,23,57,5,54],
[1,70,54,71,83,51,54,69,16,92,33,48,61,43,52,1,89,19,67,48]
]

no = 4

out = []

start_pts_nw_se = [[i, j] for i in range(len(l)-no+1) for j in range(len(l)-no+1)]
start_pts_ne_sw = [[i, j] for i in range(len(l)-no+1) for j in range(no-1, len(l))]
start_pts_down = [[i, j] for i in range(len(l)-no+1) for j in range(len(l))]
start_pts_across = [[i, j] for i in range(len(l)) for j in range(len(l)-no+1)]

for sp in start_pts_nw_se:
    m = 1
    s, e = sp[0], sp[1]
    for k in range(no):
        m *= l[s+k][e+k]
    out.append([s, e, 'diag nw se', m])
    
for sp in start_pts_ne_sw:
    m = 1
    s, e = sp[0], sp[1]
    for k in range(no):
        m *= l[s+k][e-k]
    out.append([s, e, 'diag ne sw', m])
    
for sp in start_pts_down:
    m = 1
    s, e = sp[0], sp[1]
    for k in range(no):
        m *= l[s+k][e]
    out.append([s, e, 'down', m])
    
for sp in start_pts_across:
    m = 1
    s, e = sp[0], sp[1]
    for k in range(no):
        m *= l[s][e+k]
    out.append([s, e, 'across', m])
    
sorted(out, key = lambda x: x[0], reverse=True)

# Problem 12
'''The sequence of triangle numbers is generated by adding the natural numbers. So the 7th triangle number would be 1 + 2 + 3 + 4 + 5 + 6 + 7 = 28.
The first ten terms would be: 1, 3, 6, 10, 15, 21, 28, 36, 45, 55, ...
Let us list the factors of the first seven triangle numbers:
     1: 1
     3: 1,3
     6: 1,2,3,6
    10: 1,2,5,10
    15: 1,3,5,15
    21: 1,3,7,21
    28: 1,2,4,7,14,28
We can see that 28 is the first triangle number to have over five divisors.
What is the value of the first triangle number to have over five hundred divisors?'''

# Answer 12
def triangle(n):
    return sum(np.arange(1, n + 1, 1))

def no_divisors(n):
    facts = []
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            facts.append(x)
    return len(set([n/i for i in facts] + facts)) + 2

n = 0
while True:
    n += 1
    if no_divisors(triangle(n)) > 500:
        print(triangle(n))
        break
        
# Problem 13
'''
37107287533902102798797998220837590246510135740250
46376937677490009712648124896970078050417018260538
74324986199524741059474233309513058123726617309629
91942213363574161572522430563301811072406154908250
23067588207539346171171980310421047513778063246676
89261670696623633820136378418383684178734361726757
28112879812849979408065481931592621691275889832738
44274228917432520321923589422876796487670272189318
47451445736001306439091167216856844588711603153276
70386486105843025439939619828917593665686757934951
62176457141856560629502157223196586755079324193331
64906352462741904929101432445813822663347944758178
92575867718337217661963751590579239728245598838407
58203565325359399008402633568948830189458628227828
80181199384826282014278194139940567587151170094390
35398664372827112653829987240784473053190104293586
86515506006295864861532075273371959191420517255829
71693888707715466499115593487603532921714970056938
54370070576826684624621495650076471787294438377604
53282654108756828443191190634694037855217779295145
36123272525000296071075082563815656710885258350721
45876576172410976447339110607218265236877223636045
17423706905851860660448207621209813287860733969412
81142660418086830619328460811191061556940512689692
51934325451728388641918047049293215058642563049483
62467221648435076201727918039944693004732956340691
15732444386908125794514089057706229429197107928209
55037687525678773091862540744969844508330393682126
18336384825330154686196124348767681297534375946515
80386287592878490201521685554828717201219257766954
78182833757993103614740356856449095527097864797581
16726320100436897842553539920931837441497806860984
48403098129077791799088218795327364475675590848030
87086987551392711854517078544161852424320693150332
59959406895756536782107074926966537676326235447210
69793950679652694742597709739166693763042633987085
41052684708299085211399427365734116182760315001271
65378607361501080857009149939512557028198746004375
35829035317434717326932123578154982629742552737307
94953759765105305946966067683156574377167401875275
88902802571733229619176668713819931811048770190271
25267680276078003013678680992525463401061632866526
36270218540497705585629946580636237993140746255962
24074486908231174977792365466257246923322810917141
91430288197103288597806669760892938638285025333403
34413065578016127815921815005561868836468420090470
23053081172816430487623791969842487255036638784583
11487696932154902810424020138335124462181441773470
63783299490636259666498587618221225225512486764533
67720186971698544312419572409913959008952310058822
95548255300263520781532296796249481641953868218774
76085327132285723110424803456124867697064507995236
37774242535411291684276865538926205024910326572967
23701913275725675285653248258265463092207058596522
29798860272258331913126375147341994889534765745501
18495701454879288984856827726077713721403798879715
38298203783031473527721580348144513491373226651381
34829543829199918180278916522431027392251122869539
40957953066405232632538044100059654939159879593635
29746152185502371307642255121183693803580388584903
41698116222072977186158236678424689157993532961922
62467957194401269043877107275048102390895523597457
23189706772547915061505504953922979530901129967519
86188088225875314529584099251203829009407770775672
11306739708304724483816533873502340845647058077308
82959174767140363198008187129011875491310547126581
97623331044818386269515456334926366572897563400500
42846280183517070527831839425882145521227251250327
55121603546981200581762165212827652751691296897789
32238195734329339946437501907836945765883352399886
75506164965184775180738168837861091527357929701337
62177842752192623401942399639168044983993173312731
32924185707147349566916674687634660915035914677504
99518671430235219628894890102423325116913619626622
73267460800591547471830798392868535206946944540724
76841822524674417161514036427982273348055556214818
97142617910342598647204516893989422179826088076852
87783646182799346313767754307809363333018982642090
10848802521674670883215120185883543223812876952786
71329612474782464538636993009049310363619763878039
62184073572399794223406235393808339651327408011116
66627891981488087797941876876144230030984490851411
60661826293682836764744779239180335110989069790714
85786944089552990653640447425576083659976645795096
66024396409905389607120198219976047599490197230297
64913982680032973156037120041377903785566085089252
16730939319872750275468906903707539413042652315011
94809377245048795150954100921645863754710598436791
78639167021187492431995700641917969777599028300699
15368713711936614952811305876380278410754449733078
40789923115535562561142322423255033685442488917353
44889911501440648020369068063960672322193204149535
41503128880339536053299340368006977710650566631954
81234880673210146739058568557934581403627822703280
82616570773948327592232845941706525094512325230608
22918802058777319719839450180888072429661980811197
77158542502016545090413245809786882778948721859617
72107838435069186155435662884062257473692284509516
20849603980134001723930671666823555245252804609722
53503534226472524250874054075591789781264330331690
'''
        
# Answer Problem 13
l = [37107287533902102798797998220837590246510135740250,
46376937677490009712648124896970078050417018260538,
74324986199524741059474233309513058123726617309629,
91942213363574161572522430563301811072406154908250,
23067588207539346171171980310421047513778063246676,
89261670696623633820136378418383684178734361726757,
28112879812849979408065481931592621691275889832738,
44274228917432520321923589422876796487670272189318,
47451445736001306439091167216856844588711603153276,
70386486105843025439939619828917593665686757934951,
62176457141856560629502157223196586755079324193331,
64906352462741904929101432445813822663347944758178,
92575867718337217661963751590579239728245598838407,
58203565325359399008402633568948830189458628227828,
80181199384826282014278194139940567587151170094390,
35398664372827112653829987240784473053190104293586,
86515506006295864861532075273371959191420517255829,
71693888707715466499115593487603532921714970056938,
54370070576826684624621495650076471787294438377604,
53282654108756828443191190634694037855217779295145,
36123272525000296071075082563815656710885258350721,
45876576172410976447339110607218265236877223636045,
17423706905851860660448207621209813287860733969412,
81142660418086830619328460811191061556940512689692,
51934325451728388641918047049293215058642563049483,
62467221648435076201727918039944693004732956340691,
15732444386908125794514089057706229429197107928209,
55037687525678773091862540744969844508330393682126,
18336384825330154686196124348767681297534375946515,
80386287592878490201521685554828717201219257766954,
78182833757993103614740356856449095527097864797581,
16726320100436897842553539920931837441497806860984,
48403098129077791799088218795327364475675590848030,
87086987551392711854517078544161852424320693150332,
59959406895756536782107074926966537676326235447210,
69793950679652694742597709739166693763042633987085,
41052684708299085211399427365734116182760315001271,
65378607361501080857009149939512557028198746004375,
35829035317434717326932123578154982629742552737307,
94953759765105305946966067683156574377167401875275,
88902802571733229619176668713819931811048770190271,
25267680276078003013678680992525463401061632866526,
36270218540497705585629946580636237993140746255962,
24074486908231174977792365466257246923322810917141,
91430288197103288597806669760892938638285025333403,
34413065578016127815921815005561868836468420090470,
23053081172816430487623791969842487255036638784583,
11487696932154902810424020138335124462181441773470,
63783299490636259666498587618221225225512486764533,
67720186971698544312419572409913959008952310058822,
95548255300263520781532296796249481641953868218774,
76085327132285723110424803456124867697064507995236,
37774242535411291684276865538926205024910326572967,
23701913275725675285653248258265463092207058596522,
29798860272258331913126375147341994889534765745501,
18495701454879288984856827726077713721403798879715,
38298203783031473527721580348144513491373226651381,
34829543829199918180278916522431027392251122869539,
40957953066405232632538044100059654939159879593635,
29746152185502371307642255121183693803580388584903,
41698116222072977186158236678424689157993532961922,
62467957194401269043877107275048102390895523597457,
23189706772547915061505504953922979530901129967519,
86188088225875314529584099251203829009407770775672,
11306739708304724483816533873502340845647058077308,
82959174767140363198008187129011875491310547126581,
97623331044818386269515456334926366572897563400500,
42846280183517070527831839425882145521227251250327,
55121603546981200581762165212827652751691296897789,
32238195734329339946437501907836945765883352399886,
75506164965184775180738168837861091527357929701337,
62177842752192623401942399639168044983993173312731,
32924185707147349566916674687634660915035914677504,
99518671430235219628894890102423325116913619626622,
73267460800591547471830798392868535206946944540724,
76841822524674417161514036427982273348055556214818,
97142617910342598647204516893989422179826088076852,
87783646182799346313767754307809363333018982642090,
10848802521674670883215120185883543223812876952786,
71329612474782464538636993009049310363619763878039,
62184073572399794223406235393808339651327408011116,
66627891981488087797941876876144230030984490851411,
60661826293682836764744779239180335110989069790714,
85786944089552990653640447425576083659976645795096,
66024396409905389607120198219976047599490197230297,
64913982680032973156037120041377903785566085089252,
16730939319872750275468906903707539413042652315011,
94809377245048795150954100921645863754710598436791,
78639167021187492431995700641917969777599028300699,
15368713711936614952811305876380278410754449733078,
40789923115535562561142322423255033685442488917353,
44889911501440648020369068063960672322193204149535,
41503128880339536053299340368006977710650566631954,
81234880673210146739058568557934581403627822703280,
82616570773948327592232845941706525094512325230608,
22918802058777319719839450180888072429661980811197,
77158542502016545090413245809786882778948721859617,
72107838435069186155435662884062257473692284509516,
20849603980134001723930671666823555245252804609722,
53503534226472524250874054075591789781264330331690]

str(sum(l))[:10]

# Problem 14
'''The following iterative sequence is defined for the set of positive integers:
n → n/2 (n is even)
n → 3n + 1 (n is odd)
Using the rule above and starting with 13, we generate the following sequence: 13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1
It can be seen that this sequence (starting at 13 and finishing at 1) contains 10 terms. Although it has not been proved yet (Collatz Problem), it is thought that all starting numbers finish at 1.
Which starting number, under one million, produces the longest chain?
NOTE: Once the chain starts the terms are allowed to go above one million.'''

# Answer 14
def collatz(n):
    c = 0
    while n != 1:
        if n % 2 == 0:
            n /= 2
        else:
            n = n * 3 + 1
        c += 1
    return c + 1

l = [(n, collatz(n)) for n in range(2, 1000000)]
sorted(l, key=lambda x: x[1], reverse=True)[0][0]

# MISSING ANSWER 15

# Problem 16
'''215 = 32768 and the sum of its digits is 3 + 2 + 7 + 6 + 8 = 26. What is the sum of the digits of the number 21000?'''

# Answer 16
sum([int(i) for i in str(2**1000)])

# Problem 17
'''If the numbers 1 to 5 are written out in words: one, two, three, four, five, then there are 3 + 3 + 5 + 4 + 4 = 19 letters used in total.
If all the numbers from 1 to 1000 (one thousand) inclusive were written out in words, how many letters would be used?
NOTE: Do not count spaces or hyphens. For example, 342 (three hundred and forty-two) contains 23 letters and 115 (one hundred and fifteen) contains 20 letters.
The use of "and" when writing out numbers is in compliance with British usage.'''

# Answer 17
import num2words
f = lambda x: num2words.num2words(x).replace(' ', '').replace('-', '')
s = 0
for n in range(1, 1001):
    s += len(f(n))
print(s)

# MISSING ANSWER 18

# Problem 19
'''You are given the following information, but you may prefer to do some research for yourself.
    1 Jan 1900 was a Monday.
    Thirty days has September,
    April, June and November.
    All the rest have thirty-one,
    Saving February alone,
    Which has twenty-eight, rain or shine.
    And on leap years, twenty-nine.
    A leap year occurs on any year evenly divisible by 4, but not on a century unless it is divisible by 400.
How many Sundays fell on the first of the month during the twentieth century (1 Jan 1901 to 31 Dec 2000)?'''

# Answer 19
from itertools import product

yrs = [1901 + i for i in range(100)]
mnts = [i for i in range(1, 13)]
len([i for i in list(product(yrs, mnts)) if dt.date(i[0], i[1], 1).weekday()==6])

# Problem 20
'''n! means n × (n − 1) × ... × 3 × 2 × 1
For example, 10! = 10 × 9 × ... × 3 × 2 × 1 = 3628800,
and the sum of the digits in the number 10! is 3 + 6 + 2 + 8 + 8 + 0 + 0 = 27.
Find the sum of the digits in the number 100!'''

# Answer 20
def factorial(n):
    m = 1
    for i in range(1, n+1):
        m *= i
    return m

sum([int(i) for i in str(factorial(100))])


# Problem 21
'''Let d(n) be defined as the sum of proper divisors of n (numbers less than n which divide evenly into n).
If d(a) = b and d(b) = a, where a ≠ b, then a and b are an amicable pair and each of a and b are called amicable numbers.
For example, the proper divisors of 220 are 1, 2, 4, 5, 10, 11, 20, 22, 44, 55 and 110; therefore d(220) = 284. The proper divisors of 284 are 1, 2, 4, 71 and 142; so d(284) = 220.
Evaluate the sum of all the amicable numbers under 10000.'''

# Answer 21
def sum_proper_divisors(n):
    facts = []
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            facts.append(x)
    return sum(set([1] + sorted([int(i) for i in facts + list(set([n/i for i in facts]))])))

d = {i: sum_proper_divisors(i) for i in range(0, 10000)}
sum([k for k, v in d.items() if (d[k] in d and d[v] == k and k != v)])

# MISSING ANSWER 22

# Problem 23
'''A perfect number is a number for which the sum of its proper divisors is exactly equal to the number.
For example, the sum of the proper divisors of 28 would be 1 + 2 + 4 + 7 + 14 = 28, which means that 28 is a perfect number.
A number n is called deficient if the sum of its proper divisors is less than n and it is called abundant if this sum exceeds n.
As 12 is the smallest abundant number, 1 + 2 + 3 + 4 + 6 = 16, the smallest number that can be written as the sum of two abundant numbers is 24.
By mathematical analysis, it can be shown that all integers greater than 28123 can be written as the sum of two abundant numbers.
However, this upper limit cannot be reduced any further by analysis even though it is known that the greatest number that cannot be expressed as the sum of two abundant numbers is less than this limit.
Find the sum of all the positive integers which cannot be written as the sum of two abundant numbers.'''

# Answer 23
def sum_proper_divisors(n):
    facts = []
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            facts.append(x)
    return sum(set([1] + sorted([int(i) for i in facts + list(set([n/i for i in facts]))])))

out = [i for i in range(1, 28124) if sum_proper_divisors(i) > i]
a = list(itertools.combinations(out, 2)) + [(i, i) for i in out]
b = list(set([sum(i) for i in a]))
sum([i for i in range(1, 28124) if i not in b])

# Problem 24
'''A permutation is an ordered arrangement of objects. For example, 3124 is one possible permutation of the digits 1, 2, 3 and 4.
If all of the permutations are listed numerically or alphabetically, we call it lexicographic order.
The lexicographic permutations of 0, 1 and 2 are:
012   021   102   120   201   210
What is the millionth lexicographic permutation of the digits 0, 1, 2, 3, 4, 5, 6, 7, 8 and 9?'''

# Answer 24
def lex_perm(l):
    perms = list(itertools.permutations(l))
    sort_order = lambda x: [x[i] for i in range(len(x))]
    return sorted(perms, key = sort_order)

a = lex_perm([i for i in range(10)])
int(''.join(list([str(i) for i in a[999999]])))

# Problem 25
'''The Fibonacci sequence is defined by the recurrence relation:
Fn = Fn−1 + Fn−2, where F1 = 1 and F2 = 1.
Hence the first 12 terms will be:
    F1 = 1
    F2 = 1
    F3 = 2
    F4 = 3
    F5 = 5
    F6 = 8
    F7 = 13
    F8 = 21
    F9 = 34
    F10 = 55
    F11 = 89
    F12 = 144
The 12th term, F12, is the first term to contain three digits.
What is the index of the first term in the Fibonacci sequence to contain 1000 digits?'''

# Answer 25
l = [1, 1, 2]
while len(str(l[-1])) < 1000:
    l.append(l[-1] + l[-2])
len(l)

# MISSING ANSWER 26

# Problem 27
'''Euler discovered the remarkable quadratic formula:
n2+n+41
It turns out that the formula will produce 40 primes for the consecutive integer values 0≤n≤39
. However, when n=40,402+40+41=40(40+1)+41 is divisible by 41, and certainly when n=41,412+41+41 is clearly divisible by 41.
The incredible formula n2−79n+1601
was discovered, which produces 80 primes for the consecutive values 0≤n≤79. The product of the coefficients, −79 and 1601, is −126479.
Considering quadratics of the form:
    n2+an+b
, where |a|<1000 and |b|≤1000
where |n|
is the modulus/absolute value of n
e.g. |11|=11 and |−4|=4
Find the product of the coefficients, a
and b, for the quadratic expression that produces the maximum number of primes for consecutive values of n, starting with n=0.'''

# Answer 27
from itertools import product

def isPrime(n):
    if n < 0: return False
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            return False
    return True

def f(n, a, b):
    return (n**2) + (a*n) + (b)

def total_primes(a, b):
    n = 0
    out = f(n, a, b)
    while isPrime(f(n, a, b)):
        n += 1
    return n

iters = product(np.arange(-999, 1000, 1), np.arange(-1000, 1001, 1))
out = [(i, total_primes(i[0], i[1])) for i in iters]
pair = sorted(out, key=lambda x: x[1], reverse=True)[:10][0][0]
pair[0] * pair[1]

# Problem 28
'''
Starting with the number 1 and moving to the right in a clockwise direction a 5 by 5 spiral is formed as follows:
21 22 23 24 25
20  7  8  9 10
19  6  1  2 11
18  5  4  3 12
17 16 15 14 13
It can be verified that the sum of the numbers on the diagonals is 101.
What is the sum of the numbers on the diagonals in a 1001 by 1001 spiral formed in the same way?
'''

# Answer 28
def solve(n):
    a = [i*2 for i in range(n)]
    b = sorted([0] + [i*4 for i in range(1, int(np.ceil(n/2)))]*2)
    diags = []
    s = 1
    for i in a: 
        s += i
        diags.append(s)
    s = 1
    for i in b:
        s += i
        diags.append(s)
    print(sum(diags)-1)

# Problem 29
'''Consider all integer combinations of ab for 2 ≤ a ≤ 5 and 2 ≤ b ≤ 5:
    2^2=4, 2^3=8, 2^4=16, 2^5=32
    3^2=9, 3^3=27, 3^4=81, 3^5=243
    4^2=16, 4^3=64, 4^4=256, 4^5=1024
    5^2=25, 5^3=125, 5^4=625, 5^5=3125
If they are then placed in numerical order, with any repeats removed, we get the following sequence of 15 distinct terms:
4, 8, 9, 16, 25, 27, 32, 64, 81, 125, 243, 256, 625, 1024, 3125
How many distinct terms are in the sequence generated by ab for 2 ≤ a ≤ 100 and 2 ≤ b ≤ 100?'''

# Answer 29
a = len(set([c[0]**c[1] for c in list(itertools.product(range(2, 101), range(2, 101)))]))

#Problem 30
'''Surprisingly there are only three numbers that can be written as the sum of fourth powers of their digits:
    1634 = 1^4 + 6^4 + 3^4 + 4^4
    8208 = 8^4 + 2^4 + 0^4 + 8^4
    9474 = 9^4 + 4^4 + 7^4 + 4^4
As 1 = 1^4 is not a sum it is not included.
The sum of these numbers is 1634 + 8208 + 9474 = 19316.
Find the sum of all the numbers that can be written as the sum of fifth powers of their digits.
'''

#Answer 30
upper_bound = 6*9**5
s = 0
for i in range(2, upper_bound):
    j = [int(i) for i in list(str(i))]
    if i == sum([k**5 for k in j]):
        s += i   
        
# MISSING PROBLEM 31

# Problem 32
'''
We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.
The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital.
Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.
HINT: Some products can be obtained in more than one way so be sure to only include it once in your sum.
'''

# Answer 32

l = [[str(i), str(j), str(i*j)] for i in range(1, 10000) for j in range(1, 1000)]
prods = []
for i in l:
    if ''.join([str(n) for n in sorted([int(s) for s in list(i[0] + i[1] + i[2])])]) == '123456789':
        print(i)
        prods.append(int(i[-1]))
print(sum(set(prods)))

# Problem 33
'''
The fraction 49/98 is a curious fraction, as an inexperienced mathematician in attempting to simplify it may incorrectly believe that 49/98 = 4/8, 
which is correct, is obtained by cancelling the 9s.

We shall consider fractions like, 30/50 = 3/5, to be trivial examples.

There are exactly four non-trivial examples of this type of fraction, less than one in value, and containing two digits in the numerator and denominator.

If the product of these four fractions is given in its lowest common terms, find the value of the denominator.

'''
print()
# Answer 33
c = [[i, j] for i in range(10, 100) for j in range(10, 100)]
c = [i for i in c if not (i[0]%10==0 & i[1]%10==0)]
c = [i for i in c if i[0] / i[1] < 1]
def f(x):
    if len(set(list(str(x[0]) + str(x[1])))) == 3:
        if len(set(list(str(x[0])))) != 1 and len(set(list(str(x[1])))) != 1:
            return True
    return False
c = [i for i in c if f(i)]
m = 1
for i in c:
    n = list(str(i[0]))
    d = list(str(i[1]))
    com = [x for x in n if x in d][0]
    n = [x for x in n if x!=com][0]
    d = [x for x in d if x!=com][0]
    try:
        if int(n) / int(d) == i[0] / i[1]:
            m *= (i[0] / i[1])
    except:
        pass
m

# MISSING PROBLEM 34

# Problem 35
'''
The number, 197, is called a circular prime because all rotations of the digits: 197, 971, and 719, are themselves prime.
There are thirteen such primes below 100: 2, 3, 5, 7, 11, 13, 17, 31, 37, 71, 73, 79, and 97.
How many circular primes are there below one million?
'''

# Answer 35
def is_prime(n):
    for x in range(2, int((n**.5)) + 1):
        if n % x == 0:
            return False
    return True

def f(n):
    out = []
    l = list(str(n))
    for r in range(len(l)):
        l.insert(0, l.pop())
        if not is_prime(int(''.join(l))):
            return False
    return True

sum([f(i) for i in range(2, 1000000)])

# MISSING PROBLEM 36

# MISSING PROBLEM 37

# MISSING PROBLEM 38

# MISSING PROBLEM 39

# Problem 40
'''
An irrational decimal fraction is created by concatenating the positive integers:
0.123456789101112131415161718192021...
It can be seen that the 12th digit of the fractional part is 1.
If dn represents the nth digit of the fractional part, find the value of the following expression.
d1 × d10 × d100 × d1000 × d10000 × d100000 × d1000000
'''

# Answer 40
def get_millionth():
    l = 0
    c = 0
    while l < 1000000:
        c += 1
        l += len(str(c))
    return c
millionth = get_millionth()
f = ''
for n in range(1, get_millionth()+1):
    f += str(n)
def get_n(f, n):
    return int(f[n-1])
m = 1
for i in [1, 10, 100, 1000, 10000, 100000, 1000000]:
    m *= get_n(f, i)
m